# EuroPython program grid

In [86]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 99999;
//increase max size of output area

<IPython.core.display.Javascript object>

In [87]:
import json
import datetime as dt
from random import choice, randrange, shuffle
from copy import deepcopy
from itertools import zip_longest
from functools import partial
from operator import itemgetter

from IPython.display import display, HTML
show = lambda s: display(HTML(s))

In [88]:
talk_sessions = json.load(open('accepted_talks.json'))

#all talks
all_talks = []
for s in talk_sessions.values():
    all_talks.extend(list(s.values()))

In [89]:
list(talk_sessions.keys())

['Opening session',
 'Lightning talk',
 'talk',
 'Closing session',
 'training',
 'poster',
 'EPS session']

In [90]:
#the talks worth for scheduling
grid_talks = []
sessions = talk_sessions.copy()
general_grid_sessions = ['talk', 'EPS session', 'training']
for session_name in general_grid_sessions:
    grid_talks.extend(sessions[session_name].values())
    
# remove unused fields
fields2pop = ['abstract_extra', 
              'abstract_long',
              'abstract_short',
              'twitters',
              'emails',
              'status',
              'url',
              'companies',
              'adm_type',
              'tags',
              'have_tickets',
             ]
for talk in grid_talks:
    for f in fields2pop:
        talk.pop(f)

In [91]:
# TODO: create a program grid with query-able conditions
from tinydb import TinyDB, Query

tdb = TinyDB('talks_for_grid_db.json')
for talk in grid_talks:
    tdb.insert(talk)

In [54]:
# remove talks that I know I will schedule by hand later
conditions = [('language', 'Spanish'),
              ('language', 'Basque'),
             ]

remaining_talks = []
talks2pop = []
for idx, talk in enumerate(grid_talks):
    for field, val in conditions:
        if talk[field] == val:
            remaining_talks.append(talk)
            talks2pop.append(idx)
            break

grid_talks = [talk for idx, talk in enumerate(grid_talks) if idx not in talks2pop]

In [55]:
import datetime
from   collections import OrderedDict
from   functools   import partial

tags_field = 'tag_categories'

weekday_names = [(0, 'Monday, July 18th'), 
                 (1, 'Tuesday, July 19th'),
                 (2, 'Wednesday, July 20th'),
                 (3, 'Thursday, July 21st'),
                 (4, 'Friday, July 22nd'),
                ]
weekday_names = OrderedDict(weekday_names)

room_names = [(0, 'A1' ),
              (1, 'A3' ),
              (2, 'A2' ),
              (3, 'Ba1'),
              (4, 'Ba2'),
              (5, 'E'  ),
              (6, 'A4' ),
             ]
room_names = OrderedDict(room_names)

# talk rooms
# ((h, m), conditions)
type1_schedule = [((11,  0), [('duration', 45)]), 
                  ((11, 45), [('duration', 45)]),
                  ((12, 30), [('duration', 45)]),
                  ((14, 30), [('duration', 45)]),
                  ((15, 15), [('duration', 30)]),
                  ((15, 45), [('duration', 30)]),
                  ((17,  0), [('duration', 30)]),]

type2_schedule = [((11,  0), [('duration', 45)]), 
                  ((11, 45), [('duration', 45)]),
                  ((12, 30), [('duration', 45)]),
                  ((14, 30), [('duration', 45)]),
                  ((15, 15), [('duration', 60)]),
                  ((17,  0), [('duration', 30)]),]

type3_schedule = [((11,  0), [('duration', 45)]), 
                  ((11, 45), [('duration', 45)]),
                  ((12, 30), [('duration', 30)]),
                  ((14, 30), [('duration', 60)]),
                  ((15, 15), [('duration', 30)]),
                  ((15, 45), [('duration', 30)]),
                  ((17,  0), [('duration', 30)]),]

#tutorials
type4_schedule = [((11,  0), [('duration', 180), ('type', 'Training (180 mins)')]),
                  ((13, 30), [('duration', 180), ('type', 'Training (180 mins)')])]

start_time = (9, 0) # 9:00
keynote_time   = (( 9, 15), 45)
lightning_time = ((17, 30), 60) 
lunch_time     = ((13,  0), 90) 
am_coffee_time = ((10, 30), 30)
pm_coffee_time = ((16, 15), 45) 

breaks = [(keynote_time,   'Keynote speech'),
          (lunch_time,     'Lunch'),
          (am_coffee_time, 'Coffee break'),
          (pm_coffee_time, 'Coffee break'),
          (lightning_time, 'Lightning talks')]

daily_timegrid = lambda schedule: OrderedDict([(datetime.time(*slot[0]), slot[1]) for slot in schedule])

room1_schedule = daily_timegrid(type1_schedule) 
room2_schedule = daily_timegrid(type2_schedule) 
room3_schedule = daily_timegrid(type3_schedule) 
room4_schedule = daily_timegrid(type3_schedule) 
room5_schedule = daily_timegrid(type3_schedule) 

room6_schedule = daily_timegrid(type4_schedule) 
room7_schedule = daily_timegrid(type4_schedule) 

daily_schedule = OrderedDict([(0, room1_schedule), 
                              (1, room2_schedule),
                              (2, room3_schedule),
                              (3, room4_schedule),
                              (4, room5_schedule),
                              (5, room6_schedule),
                              (6, room7_schedule)])

# [day][room] -> condition
default_condition = [('language', 'English')]
daily_conditions = {0: {5: [],
                        6: [],},
                    1: {5: [],
                        6: [],},
                    2: {3: ['language': 'Spanish'],
                        5: [],
                        6: ['language': 'Spanish'],},
                    3: {3: ['language': 'Basque']
                        5: [],
                        6: [],},
                    4: {5: [],
                        6: [],},
                   }

room_names = list(daily_schedule.keys())

# slots_durations
slots_durations = [list(slots.values()) for slots in list(daily_schedule.values())]
slots = [val for sublist in slots_durations for val in sublist]

In [56]:
def find_talk(talk_title):
    talks = []
    for talk in all_talks:
        if talk_title in talk['title']:
            talks.append(talk)
    return talks

find_talk("Hacking ético con herramientas Python")

[{'duration': 45,
  'id': 463,
  'language': 'Spanish',
  'level': 'Advanced',
  'speakers': 'Jose Manuel Ortega',
  'sub_community': '',
  'sub_title': 'Pentesting con python',
  'tag_categories': ['Security',
   'Security',
   '>>> Suggested Track',
   'Programming',
   'Security'],
  'timerange': '',
  'title': 'Hacking ético con herramientas Python',
  'track_title': '',
  'type': 'Talk (45 mins)'}]

In [57]:
tags = {}
text = []
for talk in all_talks:
    for t in talk[tags_field]:
        if t.lower() == 'python':
            continue
        if not t.lower() in tags:
            tags[t.lower()] = 0
        tags[t.lower()] += 1
        text.append(t.lower())
tags_sorted = sorted(tags.items(), key=itemgetter(1), reverse=True)

In [58]:
def pick_talk(talks, tags, duration):
    if not talks:
        raise IndexError('The list of talks is empty!')

    while tags:
        randidx = randrange(0, len(tags))
        atag    = tags.pop(randidx)[0]
        for tlkidx, talk in enumerate(talks):
            talk_tags = [t.lower() for t in talk[tags_field]]
            if talk['duration'] == duration and atag in talk_tags:
                atalk = talks.pop(tlkidx)
                return atalk

    return pick_talk_by_duration(talks, duration)


def pick_talk_by_duration(talks, duration):
    if not talks:
        raise IndexError('The list of talks is empty!')

    for tlkidx, talk in enumerate(talks):
        if talk['duration'] == duration:
            atalk = talks.pop(tlkidx)
            return atalk
    return {}

In [59]:
talks = grid_talks.copy()
shuffle(talks)

# random pick talks
week_talks = []
n_days = 5
for d in range(n_days):
    day_talks = []
    for slot in slots:
        day_tags = tags_sorted.copy()
        try:
            day_talks.append(pick_talk(talks, day_tags, slot))
        except:
            print('No talks left.')
            break
                    
    week_talks.append(day_talks)

In [60]:
weekly_schedule = OrderedDict()

for d in range(n_days):
    weekly_schedule[d] = deepcopy(daily_schedule)
    day_talks          = week_talks[d].copy()
    for room in daily_schedule:
        for slot in daily_schedule[room]:
            #print(weekly_schedule[d][room].keys())
            weekly_schedule[d][room][slot] = day_talks.pop(0)
            #print(day_talks)

In [66]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)


def tabulate(time_list, header=''):
    table = ListTable()
    table.append(header)
    for slot in time_list:
        table.append([slot] + time_list[slot])
    return table


def get_room_schedule(weekly_schedule, room_name, field='title'):
    slots = list(daily_schedule[room_name].keys())
    daily_slots = []
    for slot in slots:
        talks = [weekly_schedule[d][room_name][slot].get(field, '-') for d in range(n_days)]
        daily_slots.append((slot, talks))
    room_schedule = OrderedDict(daily_slots)
    return room_schedule


def get_day_schedule(weekly_schedule, day_num, field='title'):
    day_schedule = weekly_schedule[day_num]
    nslots = max([len(slots) for room, slots in day_schedule.items()])
    room_slots = []
    for room, talk_slots in day_schedule.items():
        room_talks = [talk.get(field, '-') for slot, talk in talk_slots.items()]
        room_slots.append(room_talks)
    schedule = OrderedDict(list(enumerate(list(map(list, zip_longest(*room_slots, fillvalue='-'))))))
    return schedule

In [67]:
if talks or remaining_talks:
    show('<h1>Not scheduled talks</h1>')
    for talk in talks + remaining_talks:
        print(talk)

{'tag_categories': ['>>> Suggested Track', '>>> Suggested Track', 'Community', 'Data Science', 'Data Science'], 'track_title': '', 'timerange': '', 'language': 'Spanish', 'type': 'Talk (45 mins)', 'level': 'Intermediate', 'sub_community': 'pydata', 'title': 'Un vector por tu palabra', 'duration': 45, 'speakers': 'Mai Giménez', 'sub_title': '“Nuevas” aproximaciones a la representación de texto para el procesamiento del lenguaje natural', 'id': 644}
{'tag_categories': ['', ''], 'track_title': '', 'timerange': '', 'language': 'Basque', 'type': 'Talk (30 mins)', 'level': 'Beginner', 'sub_community': '', 'title': 'Datu bistaratze soluzioen garapena Smartcity proiektuetan', 'duration': 30, 'speakers': 'Iker Martinez de Agirre Mendia', 'sub_title': '', 'id': 693}
{'tag_categories': ['Best Practice and Use Cases', 'Programming', 'Case Study', 'Testing', 'Testing'], 'track_title': '', 'timerange': '', 'language': 'Spanish', 'type': 'Talk (30 mins)', 'level': 'Beginner', 'sub_community': '', 'ti

# TALKS

In [68]:
for day in weekday_names:
    show('<h1>{}</h1>'.format(weekday_names[day]))
    show(tabulate(get_day_schedule(weekly_schedule, day), header=['Slot'] + room_names)._repr_html_())

Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,Building Service interfaces with OpenAPI / Swagger,Against the silos: usable encrypted email & the quest for privacy-aware services,Efficient Django,"Infrastructure as Code: ""pip install"" your environment","Beyond scraping, getting data from dynamic, heavily javascript driven, websites","So, what's all the fuss about Docker?",The Stupid Python Workshop
1,Brainwaves for hackers 3,Profiling the unprofilable,A Gentle Introduction to Neural Networks (with Python),Python in Astronomy,CloudABI: Capability based security on Linux/UNIX,Efficient Python for High-Performance Parallel Computing,Guide to make a real contribution to an open source project for novice
2,How to improve your diet and save money with Python,Effective Code Review,RESTful API - Best Practises.,Is that spam in my ham?,Writing faster Python,-,-
3,Operating on Encrypted Data with ZeroDB,TDD of Python microservices,An Introduction to Deep Learning,Modern OpenGL with Python,PSF Meeting,-,-
4,Protect your users with Circuit Breakers,A deep dive into the Pymongo MongoDB driver,Dynamic Class Generation in Python,Python Descriptors for Better Data Structures,Keeping the Lights on with Python,-,-
5,EuroPython 2017: Help us build the next edition!,Simplifying Computer Art in Python,Game Theory to the Rescue When Hard Decisions Are to Be Made,Sponsored Talk,"Python, Data & Rock'n'Roll",-,-
6,Implementing a Sound Identifier in Python,-,Writing unit tests for C code in Python,Clean code in Python,Create secure production environment using Docker,-,-


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,Raspberry Pi GPIO Zero,Jupyter for everything else,Building your own AI,Towards More Secure Emailing,Moving away from NodeJS to a pure python solution for assets,"(Machine-)Learning Chinese, with Python!","Blender: much visual, very 3d, many python."
1,Introduction to aiohttp,How OpenStack makes Python better (and vice-versa),Fast Async Code with Cython and AsyncIO,Kung Fu at Dawn with Itertools,What is the best full text search engine for Python?,Test-driven code search and the art of writing implementation-agnostic tests,"Faster Python Programs - Measure, don't Guess"
2,Do I need to switch to Go(lang) ?,Handling GPS Data with Python,Building a mBaaS framework using Django,Writing Python Native Extensions in Rust,Building a reasonably popular web application for the first time.,-,-
3,What's the point of Object Orientation?,Scaling Microservices with Crossbar.io,Nipy on functional brain MRI,-,-,-,-
4,CFFI: calling C from Python,EPS General Assembly,Ingesting 35 million hotel images with python in the cloud.,Effectively test your webapp with Python and Selenium,System Testing with pytest and docker-py,-,-
5,Behind Closed Doors: Managing Passwords in a Dangerous World,The Journey from Python Developer to Python Company Owner,MicroPython on the BBC micro:bit,MiniBrew: Brewing beer with Python,Python and Async programming,-,-
6,Making robots walk with Python,-,Python as the keystone of building and testing C++ applications,Sponsored Talk,async/await in Python 3.5 and why it is awesome,-,-


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,Asynchronous network requests in a web application,Ethical hacking with Python tools,FAT Python: a new static optimizer for Python 3.6,Analyzing Data with Python & Docker,SQLAlchemy as the backbone of a Data Science company,OpenStack Cloud Native Deployment for Application Developers,uWSGI: the full stack application server
1,Data Formats for Data Science,The Joy of Simulation: for Fun and Profit,Implementing Parallel Programming Design Patterns using EFL for Python,Things I wish I knew before starting using Python for Data Processing,Metaclasses for fun and profit: Making a declarative GUI implementation,Introduction to Deep Learning for Natural Language Processing,Python for System Administrators
2,What Python can learn from Haskell packaging,High Performance Networking in Python,Building beautiful RESTful APIs using Flask,Test Driven Deployment with Ansible 2.0,Using and abusing Python’s double-underscore methods and attributes,-,-
3,Python in Gravitational Waves Research Communities,"OMG, Bokeh is better than ever!",-,-,-,-,-
4,Learn Python The Fun Way,-,So you think your Python startup is worth $10 million...,Music transcription with Python,Writing Redis in Python with asyncio,-,-
5,"Server for IoT devices and Mobile devices using Wifi Network,",Go for Python Programmers,AWS lambda & Python,Managing Mocks,The right way to write documentation,-,-
6,Pygame Zero,-,AFP: secure cloud authentication for machines and humans.,Developing a real-time automated trading platform with Python,Pure Data and a Clean Architecture,-,-


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,The Report Of Twisted’s Death,Deep Learning with Python & TensorFlow,Get Instrumented!,-,-,Introduction to Data Wrangling,Productive Coding with PyCharm
1,"I Hate You, NLP... ;)","Automate, contribute, repeat.",It's not magic: descriptors exposed,-,-,Present-day Async Web development training: from Twisted to Tornado and AsyncIO,Manage your Python packages professionally with devpi
2,Using Service Discovery to build dynamic python applications,Designing a Pythonic Interface,Real virtual environments without virtualenv,import community,Wrestling Python into LLVM Intermediate Representation,-,-
3,Build your first OpenStack application with OpenStack PythonSDK,Performant Python,-,-,-,-,-
4,Another pair of eyes: Reviewing code well,-,How to use Metaclasses to improve your Software Design,Monkey-patching: a magic trick or a powerful tool?,"Grocker, a Python build chain for Docker",-,-
5,"Iteration, iteration, iteration",NetworkX Visualization Powered by Bokeh,Peeking into Python’s C API,Optimize Thyself,Build your Microservices with ZeroMQ,-,-
6,Get in control of your workflows with Airflow,-,Machine Learning for dummies with Python,Testing the untestable: a beginner’s guide to mock objects,Hands-on with nilearn for Neuroimaging,-,-


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,-,-,-,-,-,Property-based testing with Hypothesis,"pytest - simple, rapid and fun testing with Python"
1,-,-,-,-,-,NumPy with Cython,IPython in Depth
2,-,-,re-Discovering Python's Regular Expressions,-,-,-,-
3,-,-,-,-,-,-,-
4,Machine Learning: Power of Ensembles,-,Exploring our Python Interpreter,-,-,-,-
5,Predicting Oscar Winners & Box Office Hits with Scikit Learn,Conda - Easier Installs and Simpler Builds,Exploring Python Bytecode,-,-,-,-
6,The value of mindfulness and how it has arrived at Google,-,-,-,-,-,-


# TRAININGS

### Room E

In [69]:
tabulate(get_room_schedule(weekly_schedule, 'E'), header=['E Room'] + list(weekday_names.values()))

E Room,"Monday, July 18th","Tuesday, July 19th","Wednesday, July 20th","Thursday, July 21st","Friday, July 22nd"
11:00:00,"So, what's all the fuss about Docker?","(Machine-)Learning Chinese, with Python!",OpenStack Cloud Native Deployment for Application Developers,Introduction to Data Wrangling,Property-based testing with Hypothesis
13:30:00,Efficient Python for High-Performance Parallel Computing,Test-driven code search and the art of writing implementation-agnostic tests,Introduction to Deep Learning for Natural Language Processing,Present-day Async Web development training: from Twisted to Tornado and AsyncIO,NumPy with Cython


### Room A4

In [70]:
tabulate(get_room_schedule(weekly_schedule, 'A4'), header=['A4 Room'] + list(weekday_names.values()))

A4 Room,"Monday, July 18th","Tuesday, July 19th","Wednesday, July 20th","Thursday, July 21st","Friday, July 22nd"
11:00:00,The Stupid Python Workshop,"Blender: much visual, very 3d, many python.",uWSGI: the full stack application server,Productive Coding with PyCharm,"pytest - simple, rapid and fun testing with Python"
13:30:00,Guide to make a real contribution to an open source project for novice,"Faster Python Programs - Measure, don't Guess",Python for System Administrators,Manage your Python packages professionally with devpi,IPython in Depth


## Snippets

In [71]:
## schedules by room
# tabulate(get_room_schedule(weekly_schedule, 'A1'),  header=['A1'] + list(weekday_names.values()))
# tabulate(get_room_schedule(weekly_schedule, 'A2'),  header=['A2'] + list(weekday_names.values()))
# tabulate(get_room_schedule(weekly_schedule, 'A3'),  header=['A3'] + list(weekday_names.values()))
# tabulate(get_room_schedule(weekly_schedule, 'Ba1'), header=['Barria 1'] + list(weekday_names.values()))
# tabulate(get_room_schedule(weekly_schedule, 'Ba2'), header=['Barria 2'] + list(weekday_names.values()))